In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier, RandomForestRegressor
from econml.metalearners import TLearner, SLearner, XLearner
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import itertools
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed

In [4]:
# Data generation process- Simulation 1- linear and non linear terms in treatment effect- 80:20 treatment assignment- linear and non linera terms in baseline outcome

# Parameters
n = 10000  # Number of observations
np.random.seed(42)

# Covariates
# Age groups corresponding to cohorts
age_groups = np.random.choice([1, 2, 3, 4, 5], size=n, p=[0.2, 0.3, 0.3, 0.1, 0.1])
age_dict = {1: '1-25', 2: '26-35', 3: '36-45', 4: '46-55', 5: '56-66'}
cohort = np.array([age_dict[age] for age in age_groups])

gender = np.random.binomial(1, 0.5, n)
hh_size = np.random.randint(1, 13, n)  # Household size varies from 1 to 12
edu_level = np.random.normal(16, 2, n)

# Covariate matrix (excluding cohort since it is categorical now)
X = np.column_stack((age_groups, gender, hh_size, edu_level))

# Desired proportion of treatment group
treatment_proportion = 0.8  # 80% treatment, 20% control

# Assign treatment based on a probability distribution
W = np.random.binomial(1, treatment_proportion, n)

# Generate baseline income
baseline_income = (1500 
                   + 20 * X[:, 0]  # Linear term for age group (cohort)
                   + 50 * X[:, 1]  # Linear effect for gender
                   - 10 * X[:, 2]**2  # quadratic term for hh_size
                   + 25 * X[:, 3]  # Linear effect for edu_level
                   + np.random.normal(0, 100, n))  # Add some noise

# Generate baseline labor force status
baseline_labor_force_status = np.random.binomial(1, 0.7, n)  # 70% in labor force

# Non-linear Baseline Outcome for Income
income_0 = baseline_income
# Linear and Non-linear Treatment Effect
tau_income = (300 
              + 50 * (X[:, 2])  # Linear effect for household size
              + 20 * edu_level  # linear term
              + 10 * np.sin(edu_level)  # non-linear term
              + np.random.normal(0, 1500, n))  # Adding moderate noise

# Treated outcome
income_1 = income_0 + tau_income

# Observed outcome
income = W * income_1 + (1 - W) * income_0

# Combine data into a DataFrame
data = pd.DataFrame({
    'cohort': cohort,
    'gender': gender,
    'hh_size': hh_size,
    'edu_level': edu_level,
    'baseline_income': baseline_income,
    'baseline_labor_force_status': baseline_labor_force_status,
    'W': W,
    'income': income
})

# Display the first few rows of the dataset
print(data.head())


  cohort  gender  hh_size  edu_level  baseline_income   
0  26-35       0        6  18.341193      1652.768920  \
1  56-66       0        2  16.273134      1891.797345   
2  36-45       0        4  17.212297      1803.524148   
3  36-45       1        7  17.874516      1493.089204   
4   1-25       0       11  19.213854       618.861966   

   baseline_labor_force_status  W       income  
0                            0  0  1652.768920  
1                            0  1  1656.043264  
2                            1  1  2333.338937  
3                            1  1   335.242330  
4                            1  1  3342.256755  


In [18]:
# simulation 1 with random forest and gradient boosting regressors

# One-hot encode the cohort categorical variable
encoder = OneHotEncoder(sparse_output=False)
cohort_encoded = encoder.fit_transform(data[['cohort']])

# Combine the encoded cohort with the rest of the features
X_data = np.column_stack((cohort_encoded, data[['gender', 'hh_size', 'edu_level', 'baseline_income', 'baseline_labor_force_status']].values))

# Scale the features
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)

W_data = data['W'].values
income_0_data = income_0
income_1_data = income_1
true_tau = tau_income

# Define a function to perform a single iteration for Random Forest
def single_iteration_rf(X, W, income_0, income_1, true_tau, seed):
    np.random.seed(seed)
    W_perm = np.random.permutation(W)
    income_perm = income_1 * W_perm + income_0 * (1 - W_perm)
    sample_indices = np.random.choice(range(X.shape[0]), size=300, replace=False)
    X_train = X[sample_indices]
    W_train = W_perm[sample_indices]
    income_train = income_perm[sample_indices]

    # Use Random Forest models
    outcome_model_control = RandomForestRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10)
    outcome_model_treated = RandomForestRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10)
    Xlearner = XLearner(models=(outcome_model_control, outcome_model_treated),
                        propensity_model=LogisticRegression(max_iter=1000))
    Xlearner.fit(income_train, W_train, X=X_train)
    tau_hat = Xlearner.effect(X_train)

    # Calculate metrics
    bias = np.mean(tau_hat - true_tau[sample_indices])
    mse = np.mean((tau_hat - true_tau[sample_indices])**2)
    mae = np.mean(np.abs(tau_hat - true_tau[sample_indices]))
    ss_res = np.sum((tau_hat - true_tau[sample_indices])**2)
    ss_tot = np.sum((true_tau[sample_indices] - np.mean(true_tau[sample_indices]))**2)
    r_squared = 1 - (ss_res / ss_tot)
    explained_variance = 1 - np.var(tau_hat - true_tau[sample_indices]) / np.var(true_tau[sample_indices])
    
    return bias, mse, mae, r_squared, explained_variance

# Define a function to perform a single iteration for Gradient Boosting
def single_iteration_gb(X, W, income_0, income_1, true_tau, seed):
    np.random.seed(seed)
    W_perm = np.random.permutation(W)
    income_perm = income_1 * W_perm + income_0 * (1 - W_perm)
    sample_indices = np.random.choice(range(X.shape[0]), size=300, replace=False)
    X_train = X[sample_indices]
    W_train = W_perm[sample_indices]
    income_train = income_perm[sample_indices]

    # Use Gradient Boosting models
    outcome_model_control = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10, learning_rate=0.05)
    outcome_model_treated = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10, learning_rate=0.05)
    Xlearner = XLearner(models=(outcome_model_control, outcome_model_treated),
                        propensity_model=LogisticRegression(max_iter=1000))
    Xlearner.fit(income_train, W_train, X=X_train)
    tau_hat = Xlearner.effect(X_train)

    # Calculate metrics
    bias = np.mean(tau_hat - true_tau[sample_indices])
    mse = np.mean((tau_hat - true_tau[sample_indices])**2)
    mae = np.mean(np.abs(tau_hat - true_tau[sample_indices]))
    ss_res = np.sum((tau_hat - true_tau[sample_indices])**2)
    ss_tot = np.sum((true_tau[sample_indices] - np.mean(true_tau[sample_indices]))**2)
    r_squared = 1 - (ss_res / ss_tot)
    explained_variance = 1 - np.var(tau_hat - true_tau[sample_indices]) / np.var(true_tau[sample_indices])
    
    return bias, mse, mae, r_squared, explained_variance

# Parallelize the computation across multiple cores for Random Forest
results_rf = Parallel(n_jobs=-1)(delayed(single_iteration_rf)(X_data, W_data, income_0_data, income_1_data, true_tau, i) for i in range(50))

# Aggregate results for Random Forest
avg_bias_rf = np.mean([result[0] for result in results_rf])
avg_mse_rf = np.mean([result[1] for result in results_rf])
avg_mae_rf = np.mean([result[2] for result in results_rf])
avg_r_squared_rf = np.mean([result[3] for result in results_rf])
avg_explained_variance_rf = np.mean([result[4] for result in results_rf])

print("Random Forest Results:")
print(f"Average Bias: {avg_bias_rf}")
print(f"Average MSE: {avg_mse_rf}")
print(f"Average RMSE: {np.sqrt(avg_mse_rf)}")
print(f"Average MAE: {avg_mae_rf}")
print(f"R-squared: {avg_r_squared_rf}")
print(f"Explained Variance: {avg_explained_variance_rf}")

# Parallelize the computation across multiple cores for Gradient Boosting
results_gb = Parallel(n_jobs=-1)(delayed(single_iteration_gb)(X_data, W_data, income_0_data, income_1_data, true_tau, i) for i in range(50))

# Aggregate results for Gradient Boosting
avg_bias_gb = np.mean([result[0] for result in results_gb])
avg_mse_gb = np.mean([result[1] for result in results_gb])
avg_mae_gb = np.mean([result[2] for result in results_gb])
avg_r_squared_gb = np.mean([result[3] for result in results_gb])
avg_explained_variance_gb = np.mean([result[4] for result in results_gb])

print("\nGradient Boosting Results:")
print(f"Average Bias: {avg_bias_gb}")
print(f"Average MSE: {avg_mse_gb}")
print(f"Average RMSE: {np.sqrt(avg_mse_gb)}")
print(f"Average MAE: {avg_mae_gb}")
print(f"R-squared: {avg_r_squared_gb}")
print(f"Explained Variance: {avg_explained_variance_gb}")


Random Forest Results:
Average Bias: -0.7904896577587616
Average MSE: 15536.79811099645
Average RMSE: 124.64669314103945
Average MAE: 99.57871754046364
R-squared: 0.6168494995161107
Explained Variance: 0.6184034536938602

Gradient Boosting Results:
Average Bias: 0.530583765381042
Average MSE: 10156.924957114357
Average RMSE: 100.78157052315844
Average MAE: 80.3387450804248
R-squared: 0.7492932301314995
Explained Variance: 0.7505090655247415


In [6]:
# DGP for simulation 2 with 50-50 split in treatment assignment

# Parameters
n = 10000  # Number of observations
np.random.seed(42)

# Covariates
# Age groups corresponding to cohorts
age_groups = np.random.choice([1, 2, 3, 4, 5], size=n, p=[0.2, 0.3, 0.3, 0.1, 0.1])
age_dict = {1: '1-25', 2: '26-35', 3: '36-45', 4: '46-55', 5: '56-66'}
cohort = np.array([age_dict[age] for age in age_groups])

gender = np.random.binomial(1, 0.5, n)
hh_size = np.random.randint(1, 13, n)  # Household size varies from 1 to 12
edu_level = np.random.normal(16, 2, n)

# Covariate matrix (excluding cohort since it is categorical now)
X = np.column_stack((age_groups, gender, hh_size, edu_level))

# Desired proportion of treatment group
treatment_proportion = 0.5  # 50% treatment, 50% control

# Assign treatment based on a 50/50 probability distribution
W = np.random.binomial(1, treatment_proportion, n)

# Generate baseline income
baseline_income = (1500 
                   + 20 * X[:, 0]  # Linear term for age group (cohort)
                   + 50 * X[:, 1]  # Linear effect for gender
                   - 10 * X[:, 2]**2  # quadratic term for hh_size
                   + 25 * X[:, 3]  # Linear effect for edu_level
                   + np.random.normal(0, 100, n))  # Add some noise

# Generate baseline labor force status
baseline_labor_force_status = np.random.binomial(1, 0.7, n)  # 70% in labor force

# Non-linear Baseline Outcome for Income
income_0 = baseline_income

# Linear and Non-linear Treatment Effect
tau_income = (300 
              + 50 * (X[:, 2])  # Linear effect for household size
              + 20 * edu_level  # linear term
              + 10 * np.sin(edu_level)  # non-linear term
              + np.random.normal(0, 1500, n))  # Adding moderate noise

# Treated outcome
income_1 = income_0 + tau_income

# Observed outcome
income = W * income_1 + (1 - W) * income_0

# Combine data into a DataFrame and rename it to data_sim1
data_sim1 = pd.DataFrame({
    'cohort': cohort,
    'gender': gender,
    'hh_size': hh_size,
    'edu_level': edu_level,
    'baseline_income': baseline_income,
    'baseline_labor_force_status': baseline_labor_force_status,
    'W': W,
    'income': income
})

# Display the first few rows of the dataset
print(data_sim1.head())


  cohort  gender  hh_size  edu_level  baseline_income   
0  26-35       0        6  18.341193      1652.768920  \
1  56-66       0        2  16.273134      1891.797345   
2  36-45       0        4  17.212297      1803.524148   
3  36-45       1        7  17.874516      1493.089204   
4   1-25       0       11  19.213854       618.861966   

   baseline_labor_force_status  W       income  
0                            0  1  2307.901716  
1                            0  0  1891.797345  
2                            1  1  2333.338937  
3                            1  0  1493.089204  
4                            1  0   618.861966  


In [10]:
# SIMULATION 2 results with Random Forest and Gradient Boosting Regressors

# One-hot encode the cohort categorical variable
encoder = OneHotEncoder(sparse_output=False)
cohort_encoded = encoder.fit_transform(data_sim1[['cohort']])

# Combine the encoded cohort with the rest of the features
X_data = np.column_stack((cohort_encoded, data_sim1[['gender', 'hh_size', 'edu_level', 'baseline_income', 'baseline_labor_force_status']].values))

# Scale the features
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)

W_data = data_sim1['W'].values
income_0_data = income_0
income_1_data = income_1
true_tau = tau_income

# Define a function to perform a single iteration for Random Forest
def single_iteration_rf(X, W, income_0, income_1, true_tau, seed):
    np.random.seed(seed)
    W_perm = np.random.permutation(W)
    income_perm = income_1 * W_perm + income_0 * (1 - W_perm)
    sample_indices = np.random.choice(range(X.shape[0]), size=300, replace=False)
    X_train = X[sample_indices]
    W_train = W_perm[sample_indices]
    income_train = income_perm[sample_indices]

    # Use Random Forest models
    outcome_model_control = RandomForestRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10)
    outcome_model_treated = RandomForestRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10)
    Xlearner = XLearner(models=(outcome_model_control, outcome_model_treated),
                        propensity_model=LogisticRegression(max_iter=1000))
    Xlearner.fit(income_train, W_train, X=X_train)
    tau_hat = Xlearner.effect(X_train)

    bias = np.mean(tau_hat - true_tau[sample_indices])
    mse = np.mean((tau_hat - true_tau[sample_indices])**2)
    mae = np.mean(np.abs(tau_hat - true_tau[sample_indices]))
    ss_res = np.sum((tau_hat - true_tau[sample_indices])**2)
    ss_tot = np.sum((true_tau[sample_indices] - np.mean(true_tau[sample_indices]))**2)
    r_squared = 1 - (ss_res / ss_tot)
    explained_variance = 1 - np.var(tau_hat - true_tau[sample_indices]) / np.var(true_tau[sample_indices])
    
    return bias, mse, mae, r_squared, explained_variance

# Define a function to perform a single iteration for Gradient Boosting
def single_iteration_gb(X, W, income_0, income_1, true_tau, seed):
    np.random.seed(seed)
    W_perm = np.random.permutation(W)
    income_perm = income_1 * W_perm + income_0 * (1 - W_perm)
    sample_indices = np.random.choice(range(X.shape[0]), size=300, replace=False)
    X_train = X[sample_indices]
    W_train = W_perm[sample_indices]
    income_train = income_perm[sample_indices]

    # Use Gradient Boosting models
    outcome_model_control = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10)
    outcome_model_treated = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10)
    Xlearner = XLearner(models=(outcome_model_control, outcome_model_treated),
                        propensity_model=LogisticRegression(max_iter=1000))
    Xlearner.fit(income_train, W_train, X=X_train)
    tau_hat = Xlearner.effect(X_train)

    bias = np.mean(tau_hat - true_tau[sample_indices])
    mse = np.mean((tau_hat - true_tau[sample_indices])**2)
    mae = np.mean(np.abs(tau_hat - true_tau[sample_indices]))
    ss_res = np.sum((tau_hat - true_tau[sample_indices])**2)
    ss_tot = np.sum((true_tau[sample_indices] - np.mean(true_tau[sample_indices]))**2)
    r_squared = 1 - (ss_res / ss_tot)
    explained_variance = 1 - np.var(tau_hat - true_tau[sample_indices]) / np.var(true_tau[sample_indices])
    
    return bias, mse, mae, r_squared, explained_variance

# Parallelize the computation across multiple cores for both Random Forest and Gradient Boosting
results_rf = Parallel(n_jobs=-1)(delayed(single_iteration_rf)(X_data, W_data, income_0_data, income_1_data, true_tau, i) for i in range(50))
results_gb = Parallel(n_jobs=-1)(delayed(single_iteration_gb)(X_data, W_data, income_0_data, income_1_data, true_tau, i) for i in range(50))

# Aggregate results for Random Forest
avg_bias_rf = np.mean([result[0] for result in results_rf])
avg_mse_rf = np.mean([result[1] for result in results_rf])
avg_mae_rf = np.mean([result[2] for result in results_rf])
avg_r_squared_rf = np.mean([result[3] for result in results_rf])
avg_explained_variance_rf = np.mean([result[4] for result in results_rf])

# Aggregate results for Gradient Boosting
avg_bias_gb = np.mean([result[0] for result in results_gb])
avg_mse_gb = np.mean([result[1] for result in results_gb])
avg_mae_gb = np.mean([result[2] for result in results_gb])
avg_r_squared_gb = np.mean([result[3] for result in results_gb])
avg_explained_variance_gb = np.mean([result[4] for result in results_gb])

# Print the results in the desired format
print("Random Forest Results:")
print(f"Average Bias: {avg_bias_rf}")
print(f"Average MSE: {avg_mse_rf}")
print(f"Average RMSE: {np.sqrt(avg_mse_rf)}")
print(f"Average MAE: {avg_mae_rf}")
print(f"R-squared: {avg_r_squared_rf}")
print(f"Explained Variance: {avg_explained_variance_rf}")

print("\nGradient Boosting Results:")
print(f"Average Bias: {avg_bias_gb}")
print(f"Average MSE: {avg_mse_gb}")
print(f"Average RMSE: {np.sqrt(avg_mse_gb)}")
print(f"Average MAE: {avg_mae_gb}")
print(f"R-squared: {avg_r_squared_gb}")
print(f"Explained Variance: {avg_explained_variance_gb}")


Random Forest Results:
Average Bias: -0.9289001931183265
Average MSE: 2156765.9043037686
Average RMSE: 1468.593171815724
Average MAE: 1170.1318691605497
R-squared: 0.04675472792317272
Explained Variance: 0.05120777992062706

Gradient Boosting Results:
Average Bias: -2.7131265037175787
Average MSE: 1965815.0335941978
Average RMSE: 1402.0752596042046
Average MAE: 1074.0084055259126
R-squared: 0.1319616004288339
Explained Variance: 0.1375338586991416


In [10]:
# DGP FOR SIMULATION 3 WITH MORE COMPLEXITY IN TREATMENT EFFECT

# Parameters
n = 10000  # Number of observations
np.random.seed(42)

# Covariates
# Age groups corresponding to cohorts
age_groups = np.random.choice([1, 2, 3, 4, 5], size=n, p=[0.2, 0.3, 0.3, 0.1, 0.1])
age_dict = {1: '1-25', 2: '26-35', 3: '36-45', 4: '46-55', 5: '56-66'}
cohort = np.array([age_dict[age] for age in age_groups])

gender = np.random.binomial(1, 0.5, n)
hh_size = np.random.randint(1, 13, n)  # Household size varies from 1 to 12
edu_level = np.random.normal(16, 2, n)

# Covariate matrix (excluding cohort since it is categorical now)
X = np.column_stack((age_groups, gender, hh_size, edu_level))

# Desired proportion of treatment group
treatment_proportion = 0.8  # 80% treatment, 20% control

# Assign treatment based on a 80/20 probability distribution
W = np.random.binomial(1, treatment_proportion, n)

# Generate baseline income
baseline_income = (1500 
                   + 20 * X[:, 0]  # Linear term for age group (cohort)
                   + 50 * X[:, 1]  # Linear effect for gender
                   - 10 * X[:, 2]**2  # quadratic term for hh_size
                   + 25 * X[:, 3]  # Linear effect for edu_level
                   + np.random.normal(0, 100, n))  # Add some noise

# Generate baseline labor force status
baseline_labor_force_status = np.random.binomial(1, 0.7, n)  # 70% in labor force

# Non-linear Baseline Outcome for Income
income_0 = baseline_income

# More Non-linear Treatment Effect
tau_income = (300 
              + 50 * (X[:, 2])  # Linear effect for household size
              + 20 * edu_level  # Linear term
              + 10 * np.sin(edu_level)  # Non-linear term
              + 5 * (X[:, 2] ** 2)  # Quadratic effect for household size
              + 50 * np.log1p(X[:, 0])  # Non-linear effect for age group
              + np.random.normal(0, 1500, n))  # Adding moderate noise

# Treated outcome
income_1 = income_0 + tau_income

# Observed outcome
income = W * income_1 + (1 - W) * income_0

# Combine data into a DataFrame and rename it to data_sim3
data_sim3 = pd.DataFrame({
    'cohort': cohort,
    'gender': gender,
    'hh_size': hh_size,
    'edu_level': edu_level,
    'baseline_income': baseline_income,
    'baseline_labor_force_status': baseline_labor_force_status,
    'W': W,
    'income': income
})

# Display the first few rows of the dataset
print(data_sim3.head())


  cohort  gender  hh_size  edu_level  baseline_income   
0  26-35       0        6  18.341193      1652.768920  \
1  56-66       0        2  16.273134      1891.797345   
2  36-45       0        4  17.212297      1803.524148   
3  36-45       1        7  17.874516      1493.089204   
4   1-25       0       11  19.213854       618.861966   

   baseline_labor_force_status  W       income  
0                            0  0  1652.768920  
1                            0  1  1765.631238  
2                            1  1  2482.653655  
3                            1  1   649.557048  
4                            1  1  3981.914114  


In [16]:
# SIMULATION 3 RESULTS WITH RF AND GB REGRESSORS

# One-hot encode the cohort categorical variable
encoder = OneHotEncoder(sparse_output=False)
cohort_encoded = encoder.fit_transform(data_sim3[['cohort']])

# Combine the encoded cohort with the rest of the features
X_data = np.column_stack((cohort_encoded, data_sim3[['gender', 'hh_size', 'edu_level', 'baseline_income', 'baseline_labor_force_status']].values))

# Scale the features
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)

W_data = data_sim3['W'].values
income_0_data = income_0
income_1_data = income_1
true_tau = tau_income

# Define a function to perform a single iteration for Random Forest
def single_iteration_rf(X, W, income_0, income_1, true_tau, seed):
    np.random.seed(seed)
    W_perm = np.random.permutation(W)
    income_perm = income_1 * W_perm + income_0 * (1 - W_perm)
    sample_indices = np.random.choice(range(X.shape[0]), size=300, replace=False)
    X_train = X[sample_indices]
    W_train = W_perm[sample_indices]
    income_train = income_perm[sample_indices]

    # Use Random Forest models
    outcome_model_control = RandomForestRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10)
    outcome_model_treated = RandomForestRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10)
    Xlearner = XLearner(models=(outcome_model_control, outcome_model_treated),
                        propensity_model=LogisticRegression(max_iter=1000))
    Xlearner.fit(income_train, W_train, X=X_train)
    tau_hat = Xlearner.effect(X_train)

    bias = np.mean(tau_hat - true_tau[sample_indices])
    mse = np.mean((tau_hat - true_tau[sample_indices])**2)
    mae = np.mean(np.abs(tau_hat - true_tau[sample_indices]))
    ss_res = np.sum((tau_hat - true_tau[sample_indices])**2)
    ss_tot = np.sum((true_tau[sample_indices] - np.mean(true_tau[sample_indices]))**2)
    r_squared = 1 - (ss_res / ss_tot)
    explained_variance = 1 - np.var(tau_hat - true_tau[sample_indices]) / np.var(true_tau[sample_indices])
    
    return bias, mse, mae, r_squared, explained_variance

# Define a function to perform a single iteration for Gradient Boosting
def single_iteration_gb(X, W, income_0, income_1, true_tau, seed):
    np.random.seed(seed)
    W_perm = np.random.permutation(W)
    income_perm = income_1 * W_perm + income_0 * (1 - W_perm)
    sample_indices = np.random.choice(range(X.shape[0]), size=300, replace=False)
    X_train = X[sample_indices]
    W_train = W_perm[sample_indices]
    income_train = income_perm[sample_indices]

    # Use Gradient Boosting models
    outcome_model_control = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10) #learning_rate=0.05)
    outcome_model_treated = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10)# learning_rate=0.05)
    Xlearner = XLearner(models=(outcome_model_control, outcome_model_treated),
                        propensity_model=LogisticRegression(max_iter=1000))
    Xlearner.fit(income_train, W_train, X=X_train)
    tau_hat = Xlearner.effect(X_train)

    bias = np.mean(tau_hat - true_tau[sample_indices])
    mse = np.mean((tau_hat - true_tau[sample_indices])**2)
    mae = np.mean(np.abs(tau_hat - true_tau[sample_indices]))
    ss_res = np.sum((tau_hat - true_tau[sample_indices])**2)
    ss_tot = np.sum((true_tau[sample_indices] - np.mean(true_tau[sample_indices]))**2)
    r_squared = 1 - (ss_res / ss_tot)
    explained_variance = 1 - np.var(tau_hat - true_tau[sample_indices]) / np.var(true_tau[sample_indices])
    
    return bias, mse, mae, r_squared, explained_variance

# Parallelize the computation across multiple cores for Random Forest
results_rf = Parallel(n_jobs=-1)(delayed(single_iteration_rf)(X_data, W_data, income_0_data, income_1_data, true_tau, i) for i in range(50))

# Aggregate results for Random Forest
avg_bias_rf = np.mean([result[0] for result in results_rf])
avg_mse_rf = np.mean([result[1] for result in results_rf])
avg_mae_rf = np.mean([result[2] for result in results_rf])
avg_r_squared_rf = np.mean([result[3] for result in results_rf])
avg_explained_variance_rf = np.mean([result[4] for result in results_rf])

print("Random Forest Results:")
print(f"Average Bias: {avg_bias_rf}")
print(f"Average MSE: {avg_mse_rf}")
print(f"Average RMSE: {np.sqrt(avg_mse_rf)}")
print(f"Average MAE: {avg_mae_rf}")
print(f"R-squared: {avg_r_squared_rf}")
print(f"Explained Variance: {avg_explained_variance_rf}")

# Parallelize the computation across multiple cores for Gradient Boosting
results_gb = Parallel(n_jobs=-1)(delayed(single_iteration_gb)(X_data, W_data, income_0_data, income_1_data, true_tau, i) for i in range(50))

# Aggregate results for Gradient Boosting
avg_bias_gb = np.mean([result[0] for result in results_gb])
avg_mse_gb = np.mean([result[1] for result in results_gb])
avg_mae_gb = np.mean([result[2] for result in results_gb])
avg_r_squared_gb = np.mean([result[3] for result in results_gb])
avg_explained_variance_gb = np.mean([result[4] for result in results_gb])

print("\nGradient Boosting Results:")
print(f"Average Bias: {avg_bias_gb}")
print(f"Average MSE: {avg_mse_gb}")
print(f"Average RMSE: {np.sqrt(avg_mse_gb)}")
print(f"Average MAE: {avg_mae_gb}")
print(f"R-squared: {avg_r_squared_gb}")
print(f"Explained Variance: {avg_explained_variance_gb}")


Random Forest Results:
Average Bias: -0.7904896577587616
Average MSE: 15536.79811099645
Average RMSE: 124.64669314103945
Average MAE: 99.57871754046364
R-squared: 0.6168494995161107
Explained Variance: 0.6184034536938602

Gradient Boosting Results:
Average Bias: 0.7857682853255845
Average MSE: 9975.267975490422
Average RMSE: 99.87626332362672
Average MAE: 79.4865747511864
R-squared: 0.7537790391343141
Explained Variance: 0.7551430669120281


In [13]:
# DGP for simulation 4 with ONLY linear terms in treatment effect

import pandas as pd
import numpy as np

# Parameters
n = 10000  # Number of observations
np.random.seed(42)

# Covariates
# Age groups corresponding to cohorts
age_groups = np.random.choice([1, 2, 3, 4, 5], size=n, p=[0.2, 0.3, 0.3, 0.1, 0.1])
age_dict = {1: '1-25', 2: '26-35', 3: '36-45', 4: '46-55', 5: '56-66'}
cohort = np.array([age_dict[age] for age in age_groups])

gender = np.random.binomial(1, 0.5, n)
hh_size = np.random.randint(1, 13, n)  # Household size varies from 1 to 12
edu_level = np.random.normal(16, 2, n)

# Covariate matrix (excluding cohort since it is categorical now)
X = np.column_stack((age_groups, gender, hh_size, edu_level))

# Desired proportion of treatment group
treatment_proportion = 0.5  # 50% treatment, 50% control

# Assign treatment based on a 50/50 probability distribution
W = np.random.binomial(1, treatment_proportion, n)

# Generate baseline income
baseline_income = (1500 
                   + 20 * X[:, 0]  # Linear term for age group (cohort)
                   + 50 * X[:, 1]  # Linear effect for gender
                   - 10 * X[:, 2]**2  # Quadratic term for hh_size
                   + 25 * X[:, 3]  # Linear effect for edu_level
                   + np.random.normal(0, 100, n))  # Add some noise

# Generate baseline labor force status
baseline_labor_force_status = np.random.binomial(1, 0.7, n)  # 70% in labor force

# All linear vanilla treatment effect
tau_income = (300 
              + 50 * (X[:, 2])  # Linear effect for household size
              + 20 * edu_level  # Linear effect for education level
              + np.random.normal(0, 100, n))  # Adding minimal noise

# Treated outcome
income_1 = income_0 + tau_income

# Observed outcome
income = W * income_1 + (1 - W) * income_0

# Combine data into a DataFrame and rename it to data_sim4
data_sim4 = pd.DataFrame({
    'cohort': cohort,
    'gender': gender,
    'hh_size': hh_size,
    'edu_level': edu_level,
    'baseline_income': baseline_income,
    'baseline_labor_force_status': baseline_labor_force_status,
    'W': W,
    'income': income
})

# Display the first few rows of the dataset
print(data_sim4.head())


  cohort  gender  hh_size  edu_level  baseline_income   
0  26-35       0        6  18.341193      1652.768920  \
1  56-66       0        2  16.273134      1891.797345   
2  36-45       0        4  17.212297      1803.524148   
3  36-45       1        7  17.874516      1493.089204   
4   1-25       0       11  19.213854       618.861966   

   baseline_labor_force_status  W       income  
0                            0  1  2599.137878  
1                            0  0  1891.797345  
2                            1  1  2627.473208  
3                            1  0  1493.089204  
4                            1  0   618.861966  


In [14]:
# Results from SIMULATION 4 WITH SF AND GB REGRESSORS

# One-hot encode the cohort categorical variable
encoder = OneHotEncoder(sparse_output=False)
cohort_encoded = encoder.fit_transform(data_sim4[['cohort']])

# Combine the encoded cohort with the rest of the features
X_data = np.column_stack((cohort_encoded, data_sim4[['gender', 'hh_size', 'edu_level', 'baseline_income', 'baseline_labor_force_status']].values))

# Scale the features
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)

W_data = data_sim4['W'].values
income_0_data = income_0
income_1_data = income_1
true_tau = tau_income

# Define a function to perform a single iteration for Random Forest
def single_iteration_rf(X, W, income_0, income_1, true_tau, seed):
    np.random.seed(seed)
    W_perm = np.random.permutation(W)
    income_perm = income_1 * W_perm + income_0 * (1 - W_perm)
    sample_indices = np.random.choice(range(X.shape[0]), size=300, replace=False)
    X_train = X[sample_indices]
    W_train = W_perm[sample_indices]
    income_train = income_perm[sample_indices]

    # Use Random Forest models
    outcome_model_control = RandomForestRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10)
    outcome_model_treated = RandomForestRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10)
    Xlearner = XLearner(models=(outcome_model_control, outcome_model_treated),
                        propensity_model=LogisticRegression(max_iter=1000))
    Xlearner.fit(income_train, W_train, X=X_train)
    tau_hat = Xlearner.effect(X_train)

    bias = np.mean(tau_hat - true_tau[sample_indices])
    mse = np.mean((tau_hat - true_tau[sample_indices])**2)
    mae = np.mean(np.abs(tau_hat - true_tau[sample_indices]))
    ss_res = np.sum((tau_hat - true_tau[sample_indices])**2)
    ss_tot = np.sum((true_tau[sample_indices] - np.mean(true_tau[sample_indices]))**2)
    r_squared = 1 - (ss_res / ss_tot)
    explained_variance = 1 - np.var(tau_hat - true_tau[sample_indices]) / np.var(true_tau[sample_indices])
    
    return bias, mse, mae, r_squared, explained_variance

# Define a function to perform a single iteration for Gradient Boosting
def single_iteration_gb(X, W, income_0, income_1, true_tau, seed):
    np.random.seed(seed)
    W_perm = np.random.permutation(W)
    income_perm = income_1 * W_perm + income_0 * (1 - W_perm)
    sample_indices = np.random.choice(range(X.shape[0]), size=300, replace=False)
    X_train = X[sample_indices]
    W_train = W_perm[sample_indices]
    income_train = income_perm[sample_indices]

    # Use Gradient Boosting models
    outcome_model_control = GradientBoostingRegressor(n_estimators=100, max_depth=4, min_samples_leaf=10, learning_rate=0.05)
    outcome_model_treated = GradientBoostingRegressor(n_estimators=100, max_depth=4, min_samples_leaf=10, learning_rate=0.05)
    Xlearner = XLearner(models=(outcome_model_control, outcome_model_treated),
                        propensity_model=LogisticRegression(max_iter=1000))
    Xlearner.fit(income_train, W_train, X=X_train)
    tau_hat = Xlearner.effect(X_train)

    bias = np.mean(tau_hat - true_tau[sample_indices])
    mse = np.mean((tau_hat - true_tau[sample_indices])**2)
    mae = np.mean(np.abs(tau_hat - true_tau[sample_indices]))
    ss_res = np.sum((tau_hat - true_tau[sample_indices])**2)
    ss_tot = np.sum((true_tau[sample_indices] - np.mean(true_tau[sample_indices]))**2)
    r_squared = 1 - (ss_res / ss_tot)
    explained_variance = 1 - np.var(tau_hat - true_tau[sample_indices]) / np.var(true_tau[sample_indices])
    
    return bias, mse, mae, r_squared, explained_variance

# Parallelize the computation across multiple cores for Random Forest
results_rf = Parallel(n_jobs=-1)(delayed(single_iteration_rf)(X_data, W_data, income_0_data, income_1_data, true_tau, i) for i in range(50))

# Aggregate results for Random Forest
avg_bias_rf = np.mean([result[0] for result in results_rf])
avg_mse_rf = np.mean([result[1] for result in results_rf])
avg_mae_rf = np.mean([result[2] for result in results_rf])
avg_r_squared_rf = np.mean([result[3] for result in results_rf])
avg_explained_variance_rf = np.mean([result[4] for result in results_rf])

print("Random Forest Results:")
print(f"Average Bias: {avg_bias_rf}")
print(f"Average MSE: {avg_mse_rf}")
print(f"Average RMSE: {np.sqrt(avg_mse_rf)}")
print(f"Average MAE: {avg_mae_rf}")
print(f"R-squared: {avg_r_squared_rf}")
print(f"Explained Variance: {avg_explained_variance_rf}")

# Parallelize the computation across multiple cores for Gradient Boosting
results_gb = Parallel(n_jobs=-1)(delayed(single_iteration_gb)(X_data, W_data, income_0_data, income_1_data, true_tau, i) for i in range(50))

# Aggregate results for Gradient Boosting
avg_bias_gb = np.mean([result[0] for result in results_gb])
avg_mse_gb = np.mean([result[1] for result in results_gb])
avg_mae_gb = np.mean([result[2] for result in results_gb])
avg_r_squared_gb = np.mean([result[3] for result in results_gb])
avg_explained_variance_gb = np.mean([result[4] for result in results_gb])

print("\nGradient Boosting Results:")
print(f"Average Bias: {avg_bias_gb}")
print(f"Average MSE: {avg_mse_gb}")
print(f"Average RMSE: {np.sqrt(avg_mse_gb)}")
print(f"Average MAE: {avg_mae_gb}")
print(f"R-squared: {avg_r_squared_gb}")
print(f"Explained Variance: {avg_explained_variance_gb}")


Random Forest Results:
Average Bias: 0.7899569959458497
Average MSE: 11822.6325362386
Average RMSE: 108.73192969978322
Average MAE: 86.39272826540237
R-squared: 0.7082312384800445
Explained Variance: 0.7100623202177975

Gradient Boosting Results:
Average Bias: 0.30331393987323746
Average MSE: 9718.699758558865
Average RMSE: 98.58346594920907
Average MAE: 77.51552856619497
R-squared: 0.7601316374955205
Explained Variance: 0.7618391895129841


In [20]:
# Simulation 1 WITH T-LEARNER- random forests and gradient boosting base learners

# One-hot encode the cohort categorical variable
encoder = OneHotEncoder(sparse_output=False)
cohort_encoded = encoder.fit_transform(data[['cohort']])

# Combine the encoded cohort with the rest of the features
X_data = np.column_stack((cohort_encoded, data[['gender', 'hh_size', 'edu_level', 'baseline_income', 'baseline_labor_force_status']].values))

# Scale the features
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)

W_data = data['W'].values
income_0_data = income_0
income_1_data = income_1
true_tau = tau_income

# Define a function to perform a single iteration for Random Forest using TLearner
def single_iteration_rf_tlearner(X, W, income_0, income_1, true_tau, seed):
    np.random.seed(seed)
    sample_indices = np.random.choice(range(X.shape[0]), size=300, replace=False)
    X_train = X[sample_indices]
    W_train = W[sample_indices]
    income_train = income_1[sample_indices] * W_train + income_0[sample_indices] * (1 - W_train)

    # Use Random Forest models
    outcome_model = RandomForestRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10)
    tlearner = TLearner(models=outcome_model)
    tlearner.fit(Y=income_train, T=W_train, X=X_train)
    tau_hat = tlearner.effect(X_train)

    # Calculate metrics using the sampled portion of true_tau
    bias = np.mean(tau_hat - true_tau[sample_indices])
    mse = np.mean((tau_hat - true_tau[sample_indices])**2)
    mae = np.mean(np.abs(tau_hat - true_tau[sample_indices]))
    ss_res = np.sum((tau_hat - true_tau[sample_indices])**2)
    ss_tot = np.sum((true_tau[sample_indices] - np.mean(true_tau[sample_indices]))**2)
    r_squared = 1 - (ss_res / ss_tot)
    explained_variance = 1 - np.var(tau_hat - true_tau[sample_indices]) / np.var(true_tau[sample_indices])
    
    return bias, mse, mae, r_squared, explained_variance

# Define a function to perform a single iteration for Gradient Boosting using TLearner
def single_iteration_gb_tlearner(X, W, income_0, income_1, true_tau, seed):
    np.random.seed(seed)
    sample_indices = np.random.choice(range(X.shape[0]), size=300, replace=False)
    X_train = X[sample_indices]
    W_train = W[sample_indices]
    income_train = income_1[sample_indices] * W_train + income_0[sample_indices] * (1 - W_train)

    # Use Gradient Boosting models
    outcome_model = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=10, learning_rate=0.05)
    tlearner = TLearner(models=outcome_model)
    tlearner.fit(Y=income_train, T=W_train, X=X_train)
    tau_hat = tlearner.effect(X_train)

    # Calculate metrics using the sampled portion of true_tau
    bias = np.mean(tau_hat - true_tau[sample_indices])
    mse = np.mean((tau_hat - true_tau[sample_indices])**2)
    mae = np.mean(np.abs(tau_hat - true_tau[sample_indices]))
    ss_res = np.sum((tau_hat - true_tau[sample_indices])**2)
    ss_tot = np.sum((true_tau[sample_indices] - np.mean(true_tau[sample_indices]))**2)
    r_squared = 1 - (ss_res / ss_tot)
    explained_variance = 1 - np.var(tau_hat - true_tau[sample_indices]) / np.var(true_tau[sample_indices])
    
    return bias, mse, mae, r_squared, explained_variance

# Parallelize the computation across multiple cores for Random Forest using TLearner
results_rf = Parallel(n_jobs=-1)(delayed(single_iteration_rf_tlearner)(X_data, W_data, income_0_data, income_1_data, true_tau, i) for i in range(50))

# Aggregate results for Random Forest
avg_bias_rf = np.mean([result[0] for result in results_rf])
avg_mse_rf = np.mean([result[1] for result in results_rf])
avg_mae_rf = np.mean([result[2] for result in results_rf])
avg_r_squared_rf = np.mean([result[3] for result in results_rf])
avg_explained_variance_rf = np.mean([result[4] for result in results_rf])

print("Random Forest Results with TLearner:")
print(f"Average Bias: {avg_bias_rf}")
print(f"Average MSE: {avg_mse_rf}")
print(f"Average RMSE: {np.sqrt(avg_mse_rf)}")
print(f"Average MAE: {avg_mae_rf}")
print(f"R-squared: {avg_r_squared_rf}")
print(f"Explained Variance: {avg_explained_variance_rf}")

# Parallelize the computation across multiple cores for Gradient Boosting using TLearner
results_gb = Parallel(n_jobs=-1)(delayed(single_iteration_gb_tlearner)(X_data, W_data, income_0_data, income_1_data, true_tau, i) for i in range(50))

# Aggregate results for Gradient Boosting
avg_bias_gb = np.mean([result[0] for result in results_gb])
avg_mse_gb = np.mean([result[1] for result in results_gb])
avg_mae_gb = np.mean([result[2] for result in results_gb])
avg_r_squared_gb = np.mean([result[3] for result in results_gb])
avg_explained_variance_gb = np.mean([result[4] for result in results_gb])

print("\nGradient Boosting Results with TLearner:")
print(f"Average Bias: {avg_bias_gb}")
print(f"Average MSE: {avg_mse_gb}")
print(f"Average RMSE: {np.sqrt(avg_mse_gb)}")
print(f"Average MAE: {avg_mae_gb}")
print(f"R-squared: {avg_r_squared_gb}")
print(f"Explained Variance: {avg_explained_variance_gb}")


Random Forest Results with TLearner:
Average Bias: -3.5925431837471753
Average MSE: 33215.630561839105
Average RMSE: 182.251558462031
Average MAE: 141.11276330253747
R-squared: 0.18645611815977609
Explained Variance: 0.19511416162750442

Gradient Boosting Results with TLearner:
Average Bias: -4.131573423901446
Average MSE: 11209.205225018752
Average RMSE: 105.87353411036561
Average MAE: 78.47263967474576
R-squared: 0.7253406496064986
Explained Variance: 0.7294113375554717


In [4]:
# s LEARNER ON SIMULATION 1 WITH RANDOM FOREST AND GRADIENT BOOSTING REGRESSORS

# One-hot encode the cohort categorical variable
encoder = OneHotEncoder(sparse_output=False)
cohort_encoded = encoder.fit_transform(data[['cohort']])

# Combine the encoded cohort with the rest of the features
X_data = np.column_stack((
    cohort_encoded, 
    data[['gender', 'hh_size', 'edu_level', 'baseline_income', 'baseline_labor_force_status']].values
))

# Scale the features
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)

# The treatment variable
W = data['W'].values

# Observed income (the outcome)
y = data['income'].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test, W_train, W_test = train_test_split(X_data, y, W, test_size=0.3, random_state=42)

# Combine the features with the treatment for the S-learner
X_train_with_treatment = np.column_stack((X_train, W_train))
X_test_with_treatment = np.column_stack((X_test, W_test))

# Train Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_with_treatment, y_train)
y_pred_rf = rf.predict(X_test_with_treatment)

# Train Gradient Boosting model
gb = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb.fit(X_train_with_treatment, y_train)
y_pred_gb = gb.predict(X_test_with_treatment)

# Calculate performance metrics for Random Forest
bias_rf = np.mean(y_pred_rf - y_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
explained_variance_rf = explained_variance_score(y_test, y_pred_rf)

# Calculate performance metrics for Gradient Boosting
bias_gb = np.mean(y_pred_gb - y_test)
mse_gb = mean_squared_error(y_test, y_pred_gb)
rmse_gb = np.sqrt(mse_gb)
mae_gb = mean_absolute_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)
explained_variance_gb = explained_variance_score(y_test, y_pred_gb)

# Display results
print("Random Forest Results:")
print(f"Average Bias: {bias_rf}")
print(f"Average MSE: {mse_rf}")
print(f"Average RMSE: {rmse_rf}")
print(f"Average MAE: {mae_rf}")
print(f"R-squared: {r2_rf}")
print(f"Explained Variance: {explained_variance_rf}")

print("\nGradient Boosting Results:")
print(f"Average Bias: {bias_gb}")
print(f"Average MSE: {mse_gb}")
print(f"Average RMSE: {rmse_gb}")
print(f"Average MAE: {mae_gb}")
print(f"R-squared: {r2_gb}")
print(f"Explained Variance: {explained_variance_gb}")


Random Forest Results:
Average Bias: 41.53389767564463
Average MSE: 1982648.61464069
Average RMSE: 1408.065557650172
Average MAE: 1012.2734478684324
R-squared: 0.03290909371285766
Explained Variance: 0.03375054102196817

Gradient Boosting Results:
Average Bias: 26.725320310990405
Average MSE: 1804728.8554881464
Average RMSE: 1343.401970926106
Average MAE: 971.7320164955144
R-squared: 0.11969430610734244
Explained Variance: 0.12004269747592466
